## Import the scene graph file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Authenticate
drive = None
def authenticate():
  global drive
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
  authenticate()
  
  for fileId in fileIds:    
    
    downloaded = drive.CreateFile({"id": fileId[1]})
    downloaded.GetContentFile(fileId[0])

In [ ]:
#Do not downloading training and validation dataset at same time 
try:
  _ = open("publaynet_full_feature_sg_train.pkl", "r")
except:
  downloadFiles([["publaynet_full_feature_sg_train.pkl", "1-2YRfGl8kFXUvXwAcNJ_n4o_wqOcc2oo"]])

In [ ]:
# downloading the scene graph file
import pickle
with open('publaynet_full_feature_sg_train.pkl', 'rb') as f:
    train_list_dict=pickle.load(f)

In [ ]:
try:
  _ = open("publaynet_full_feature_sg_test.json", "r")
except:
  downloadFiles([["publaynet_full_feature_sg_test.json", "1-2pZw_h_NbqE5jHXOOFdJFxwTUl77OE4"]])

In [ ]:
with open('publaynet_full_feature_sg_test.json', 'r') as f:
    eval_list_dict=json.load(f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Publaynet/new_train_0_7_text_visual_localid.pkl','rb') as f:
  train_list_dict = pickle.load(f)

In [ ]:
print(train_list_dict.keys())
print(train_list_dict['282944']['objects'].keys())

dict_keys(['282944', '285236', '284513', '285613', '282966', '254597', '285483', '282973', '285257', '285511', '286052', '286059', '282987', '286185', '282994', '286192', '285299', '284597', '285541', '286199', '286207', '284604', '286214', '284919', '285549', '284940', '284625', '285124', '284956', '284639', '283009', '283016', '285592', '284667', '285669', '286417', '283038', '286424', '286431', '283052', '286438', '286445', '286508', '285399', '286452', '287671', '287678', '287493', '285350', '287451', '288065', '288826', '287699', '288791', '288742', '288749', '289026', '286566', '285166', '289314', '290010', '287734', '289892', '290038', '289836', '287741', '289899', '289913', '289921', '288924', '288931', '290685', '288946', '290755', '288953', '290275', '288156', '289950', '290804', '290523', '287762', '290839', '290811', '288974', '290846', '290818', '290296', '288170', '290825', '290832', '289989', '290317', '288184', '291167', '291174', '290331', '290790', '288198', '291442',

In [ ]:
print(eval_list_dict.keys())
print(eval_list_dict['284112']['objects']['0']['text_density'])
print(eval_list_dict['284112']['objects']['2'])
print(eval_list_dict['284112']['objects']['1'].keys())

## Define some pre-processing function

### Set **k** nearest neighbour for distance base weighted graph
This is used to generate visual feature and text density based graph embedding, to generate a new key for object dictionary named 'near gap'.

In [ ]:
# selected top k gap distance features
def top_k_nearest(gap_dict, k):
  new_gap_dict = {}
  try:
    list1= sorted(gap_dict.values())
  except:
    print(gap_dict)
  top_k_value = list1[:k]
  for obj in gap_dict:
    if gap_dict[obj] in top_k_value:
      new_gap_dict[obj] = gap_dict[obj]
  return new_gap_dict

In [ ]:
print(train_list_dict.keys())

dict_keys(['287233', '287535', '286515', '286853', '287465', '287240', '286909', '286867', '286523', '286874', '287931', '287486', '287283', '287304', '287521', '287311', '285152', '287573', '288457', '288756', '287980', '288488', '289054', '211687', '286559', '288763', '288523', '289082', '287367', '288100', '288401', '288770', '289119', '289577', '287727', '288600', '289724', '289606', '289829', '289420', '290045', '289621', '289629', '289492', '288910', '289745', '288917', '289752', '288672', '289935', '289664', '290748', '289671', '289942', '290168', '289696', '289513', '290720', '288995', '289002', '287776', '289527', '290003', '290881', '290551', '289243', '291421', '291132', '290474', '290572', '291890', '287804', '291673', '290923', '291897', '290944', '292483', '291680', '290958', '292279', '291904', '292696', '291052', '292286', '290608', '292703', '292023', '290615', '291911', '292843', '291060', '290629', '292413', '291953', '291067', '292051', '292888', '292420', '291076',

In [ ]:
print(train_list_dict['282944']['objects'].keys())
print(train_list_dict['282944']['objects']['3']['gap'])

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8'])
{'0': 11.150000000000006, '1': 11.169999999999987, '2': 11.169999999999987, '4': 1.3100000000000023, '5': 115.84000000000006, '6': 207.36000000000004, '7': 241.92, '8': 193.28}


In [ ]:
# default k is 2
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    new_gap_dict = top_k_nearest(train_list_dict[img]['objects'][obj]['gap'],2)
    train_list_dict[img]['objects'][obj]['near_gap'] = new_gap_dict

In [ ]:
for img in eval_list_dict:
  for obj in eval_list_dict[img]['objects']:
    new_gap_dict = top_k_nearest(eval_list_dict[img]['objects'][obj]['gap'],2)
    eval_list_dict[img]['objects'][obj]['near_gap'] = new_gap_dict

### Some Searching Tools

In [ ]:
# according to id to search text
bbox_id_text = {}
for l in train_list_dict:
  for obj in train_list_dict[l]['objects']:
    id = train_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = train_list_dict[l]['objects'][obj]['text']

In [ ]:
bbox_id_text = {}
for l in eval_list_dict:
  for obj in eval_list_dict[l]['objects']:
    id = eval_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = eval_list_dict[l]['objects'][obj]['text']

In [ ]:
# according to id to search object id in a document
def globalid_to_localid(id):
  for l in train_list_dict:
    for obj in train_list_dict[l]['objects']:
      if id == train_list_dict[l]['objects'][obj]['id']:
        return obj
  for l in eval_list_dict:
    for obj in eval_list_dict[l]['objects']:
      if id == eval_list_dict[l]['objects'][obj]['id']:
        return obj

In [ ]:
id_density_dict = {}
for l in train_list_dict:
  id_density_dict[l] = {}
  for obj in train_list_dict[l]['objects']:
    try:
      id_density_dict[l][obj] = train_list_dict[l]['objects'][obj]['visual_feature']
    except:
      id_density_dict[l][obj] = [0]*2048

In [ ]:
# this dictionary used to transfer object id to text_density/text_number/char_density/char_number/visual_embedding/
id_density_dict = {}
for l in eval_list_dict:
  id_density_dict[l] = {}
  for obj in eval_list_dict[l]['objects']:
    try:
      id_density_dict[l][obj] = eval_list_dict[l]['objects'][obj]['visual_feature']
    except:
      id_density_dict[l][obj] = [0]*2048

### Change Scalar to High Dimensional Vectors

In [ ]:
import math
import torch
# one dimensional feature embedding
def positionalencoding1d(d_model, feature_list):
    """
    :param d_model: dimension of the model
    :param feature_list: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(len(feature_list), d_model)
    feats = torch.tensor(feature_list).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(feats.float() * div_term)
    pe[:, 1::2] = torch.cos(feats.float() * div_term)
    pe = pe.tolist()
    return pe


## Extracting Information from training and evaluation json file

### For Gap Distance weighted GCN model

In [ ]:
# Generate local graph based training dataset objects and relations list
new_train_list_dict = {}
for l in train_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []
  
  for obj in train_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    tem_rel = [obj,obj,0]
    tem_dic['relationships'].append(tem_rel)
    for rel in train_list_dict[l]['objects'][obj]['near_gap']:
      tem_rel = [obj,rel]
      tem_rel.append(train_list_dict[l]['objects'][obj]['near_gap'][rel])
      tem_dic['relationships'].append(tem_rel)
  new_train_list_dict[l] = tem_dic

In [ ]:
# Generate local graph based validation dataset objects and relations list
new_eval_list_dict = {}
for l in eval_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []

  for obj in eval_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    tem_rel = [obj,obj,0]
    tem_dic['relationships'].append(tem_rel)
    for rel in eval_list_dict[l]['objects'][obj]['near_gap']:
      tem_rel = [obj,rel]
      tem_rel.append(eval_list_dict[l]['objects'][obj]['near_gap'][rel])
      tem_dic['relationships'].append(tem_rel)
  new_eval_list_dict[l] = tem_dic

In [ ]:
#Using for searching real node size
num_obj_dict = {}
for l in new_train_list_dict:
    num_obj_dict[l] = len(new_train_list_dict[l]['objects'])

In [ ]:
num_obj_dict = {}
for l in new_eval_list_dict:
    num_obj_dict[l] = len(new_eval_list_dict[l]['objects'])

In [ ]:
max = 0
len_list = []
for l in num_obj_dict:
  len_list.append(num_obj_dict[l])
  if max < num_obj_dict[l]:
    max = num_obj_dict[l]
print(max)

84


In [ ]:
# useless code only rename
list_dict_train={}
for l in new_train_list_dict:
    list_dict_train[l] = new_train_list_dict[l]

In [ ]:
list_dict_test = {}
for l in new_eval_list_dict:
    list_dict_test[l] = new_eval_list_dict[l]

## Training and Validation dataframe generation

In [ ]:
#generate the training and labeling information for each image
#this information can be used to transfer object id to corresponding visual, text density or other kinds of features.
density_list_train = []
label_list_train = []
img_list_train = []
label_dict_train = {}
for l in train_list_dict:
  label_dict_train[l] = []
  for obj in train_list_dict[l]['objects']:
    density_list_train.append(obj)
    label_list_train.append(train_list_dict[l]['objects'][obj]['category_id'])
    img_list_train.append(l)
    label_dict_train[l].append(train_list_dict[l]['objects'][obj]['category_id'])
  for obj in range(num_obj_dict[l],85):
    density_list_train.append(str(obj))
    label_list_train.append(str(-1))
    img_list_train.append(l)
    label_dict_train[l].append(str(-1))

In [ ]:
# generating evalutaion labeling informaiton dictionary
density_list_eval = []
label_list_eval = []
label_dict_eval = {}
img_list_eval = []
for l in eval_list_dict:
  label_dict_eval[l] = []
  for obj in eval_list_dict[l]['objects']:
    density_list_eval.append(obj)
    label_list_eval.append(eval_list_dict[l]['objects'][obj]['category_id'])
    img_list_eval.append(l)
    label_dict_eval[l].append(eval_list_dict[l]['objects'][obj]['category_id'])
  for obj in range(num_obj_dict[l],85):
    density_list_eval.append(str(obj))
    label_list_eval.append(str(-1))
    img_list_eval.append(l)
    label_dict_eval[l].append(str(-1))

In [ ]:
from pandas import DataFrame
df_train = DataFrame(density_list_train,columns=['density'])
df_train['label'] = label_list_train
df_train['image'] = img_list_train

In [ ]:
from pandas import DataFrame
df_eval = DataFrame(density_list_eval,columns=['density'])
df_eval['label'] = label_list_eval
df_eval['image'] = img_list_eval

NameError: ignored

In [ ]:
df_train.head(85)

,density,label,image
0,0,1,282944
1,1,1,282944
2,2,1,282944
3,3,1,282944
4,4,1,282944
...,...,...,...
80,80,-1,282944
81,81,-1,282944
82,82,-1,282944
83,83,-1,282944


In [ ]:
label_list_train = df_train['label'].fillna('other').tolist()
df_train_clean = df_train[df_train['label'].notnull()][['density','label','image']]

In [ ]:
label_list_eval = df_eval['label'].fillna('other').tolist()
df_eval_clean = df_eval[df_eval['label'].notnull()][['density','label','image']] 

NameError: ignored

In [ ]:
df_train_clean.head()

,density,label,image
0,0,1,282944
1,1,1,282944
2,2,1,282944
3,3,1,282944
4,4,1,282944


In [ ]:
print(list_dict_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
label_list_train = df_train_clean['label'].to_list()
id_list_train = df_train_clean['density'].to_list()
img_list_train = df_train_clean['image'].to_list()


In [ ]:
label_list_eval = df_eval_clean['label'].to_list()
id_list_eval = df_eval_clean['density'].to_list()
img_list_eval = df_eval_clean['image'].to_list()

In [ ]:
obj_list_train = []
for i in range(len(id_list_train)):
  tem = []
  tem = [id_list_train[i],img_list_train[i]]
  obj_list_train.append(tem)

In [ ]:
obj_list_eval = []
for i in range(len(id_list_eval)):
  tem = []
  tem = [id_list_eval[i],img_list_eval[i]]
  obj_list_eval.append(tem)

In [ ]:
print(len(obj_list_train))

510000


## Convert label into one-hot

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
num_class = len(list(set(label_list_train)))
lEnc = LabelEncoder()
lEnc.fit(np.unique(list(set(label_list_train))))
labels_one_hot_train = {}
for f in label_dict_train:
  num_labels = lEnc.transform(label_dict_train[f])
  labels_one_hot_train[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_train[f].append(to_add)


In [ ]:
labels_one_hot_eval = {}
for f in label_dict_eval:
  num_labels = lEnc.transform(label_dict_eval[f])
  labels_one_hot_eval[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_eval[f].append(to_add)

In [ ]:
print(len(labels_one_hot_train))

30000


## Build Graph

Build empty node list of **each image** for training and validation data

In [ ]:
node_lists_train = {}
for l in new_train_list_dict:
  node_lists_train[l] = []
  for obj in new_train_list_dict[l]['objects']:
    node_lists_train[l].append(obj)


In [ ]:
node_lists_eval = {}
for l in new_eval_list_dict:
  node_lists_eval[l] = []
  for obj in new_eval_list_dict[l]['objects']:
    node_lists_eval[l].append(obj)

Build empty local graph for each image. Each object key contains two empty list to store gap and related other objects

In [ ]:
# build local graph
object_graph_dict_train = {}
for l in new_train_list_dict:
  object_graph_dict_train[l] = {}
  for obj in new_train_list_dict[l]['objects']:
    object_graph_dict_train[l][obj] = {}
    object_graph_dict_train[l][obj]['gap'] = []
    object_graph_dict_train[l][obj]['obj2'] = []


In [ ]:
object_graph_dict_eval = {}
for l in new_eval_list_dict:
  object_graph_dict_eval[l] = {}
  for obj in new_eval_list_dict[l]['objects']:
    object_graph_dict_eval[l][obj] = {}
    object_graph_dict_eval[l][obj]['gap'] = []
    object_graph_dict_eval[l][obj]['obj2'] = []

Fill up the empty graph based on scene graph json file

In [ ]:
#fill the empty graph
for sg in list_dict_train:
    obj_list = list_dict_train[sg]['objects']
    for rel_item in list_dict_train[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_train[sg][rel_item[0]]['obj2'].append(obj2)
        object_graph_dict_train[sg][rel_item[0]]['gap'].append(rel_item[2])

In [ ]:
#fill the empty graph
for sg in list_dict_test:
    obj_list = list_dict_test[sg]['objects']
    for rel_item in list_dict_test[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_eval[sg][rel_item[0]]['obj2'].append(obj2)
        object_graph_dict_eval[sg][rel_item[0]]['gap'].append(rel_item[2])

In [ ]:
print(object_graph_dict_train['100247'])

{'0': {'gap': [0, 18.01000000000002, 7.780000000000001], 'obj2': ['0', '6', '9']}, '1': {'gap': [0, 37.26000000000002, 37.26000000000002, 25.18999999999999], 'obj2': ['1', '0', '9', '10']}, '2': {'gap': [0, 2.610000000000042, 7.800000000000011], 'obj2': ['2', '4', '10']}, '3': {'gap': [0, 17.829999999999984, 16.970000000000056], 'obj2': ['3', '4', '11']}, '4': {'gap': [0, 2.610000000000042, 17.819999999999993, 17.819999999999993], 'obj2': ['4', '2', '5', '11']}, '5': {'gap': [0, 17.819999999999993, 7.739999999999913], 'obj2': ['5', '4', '11']}, '6': {'gap': [0, 18.01000000000002, 18.01000000000002], 'obj2': ['6', '0', '9']}, '7': {'gap': [0, 72.46999999999997, 8.13999999999998, 72.46999999999997], 'obj2': ['7', '5', '8', '11']}, '8': {'gap': [0, 8.13999999999998, 93.35], 'obj2': ['8', '7', '9']}, '9': {'gap': [0, 7.780000000000001, 18.01000000000002], 'obj2': ['9', '0', '6']}, '10': {'gap': [0, 25.18999999999999, 7.800000000000011], 'obj2': ['10', '1', '2']}, '11': {'gap': [0, 16.97000

In [ ]:
node_size_train = []
for f in object_graph_dict_train:
  l = object_graph_dict_train[f].keys()
  node_size_train.append(len(l))

In [ ]:

node_size_eval = []
for f in object_graph_dict_eval:
  l = object_graph_dict_eval[f].keys()
  node_size_eval.append(len(l))

In [ ]:
max_len = 85
col_train = {}
row_train = {}
weight_train = {}
for f in object_graph_dict_train:
  col_train[f] = []
  row_train[f] = []
  weight_train[f] = []
  for obj in object_graph_dict_train[f]:
    obj_rel_list = object_graph_dict_train[f][obj]['obj2']
    for obj2 in object_graph_dict_train[f]:
      if obj2 in obj_rel_list:
        for i in range(len(obj_rel_list)):
          if obj2 == obj_rel_list[i]:
            weight_train[f].append(1/(object_graph_dict_train[f][obj]['gap'][i]+1000))
            row_train[f].append(obj)
            col_train[f].append(obj_rel_list[i])
      else:
        weight_train[f].append(0.0)
        row_train[f].append(obj)
        col_train[f].append(obj2)


In [ ]:
max_len = 85
col_eval = {}
row_eval = {}
weight_eval = {}
for f in object_graph_dict_eval:
  col_eval[f] = []
  row_eval[f] = []
  weight_eval[f] = []
  for obj in object_graph_dict_eval[f]:
    obj_rel_list = object_graph_dict_eval[f][obj]['obj2']
    for i in range(len(obj_rel_list)):
      try:
        weight_eval[f].append(1/object_graph_dict_eval[f][obj]['gap'][i])
      except:
        weight_eval[f].append(0.0)
      row_eval[f].append(obj)
      col_eval[f].append(obj_rel_list[i])

In [ ]:
#PROBLEMS
# how to determine the weight to the node itself
# how to padding the node

In [ ]:
import scipy.sparse as sp
node_size = 85
adj_train = {}
for f in weight_train:
  adj_train[f] = sp.csr_matrix((weight_train[f], (row_train[f], col_train[f])), shape=(node_size, node_size))

In [ ]:
import scipy.sparse as sp
node_size = 85
adj_eval = {}
for f in weight_eval:
  adj_eval[f] = sp.csr_matrix((weight_eval[f], (row_eval[f], col_eval[f])), shape=(node_size, node_size))

##Build Model

In [ ]:
# from inits import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])



class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)
            else:
                pre_sup = self.vars['weights_' + str(i)]            
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']
			
		
        self.embedding = output #output
        return self.act(output)


In [ ]:
# from metrics import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)

In [ ]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])
        self.pred = tf.argmax(self.outputs, 1)
        self.labels = tf.argmax(self.placeholders['labels'], 1)

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            featureless=False,
                                            sparse_inputs=True,
                                            logging=self.logging))
        
        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x, #
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [ ]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    print(preds)
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))

    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

## Model Setting

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow.compat.v1 as tf

from sklearn import metrics
import random
import os
import sys


# # Set random seed
# seed = random.randint(1, 200)
# np.random.seed(seed)
# tf.set_random_seed(seed)


# Settings
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

flags = tf.app.flags
FLAGS = flags.FLAGS
for name in list(flags.FLAGS):
      delattr(flags.FLAGS,name)
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', 'sencegraph', 'Dataset string.')
# 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_string('model', 'gcn', 'Model string.')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 10, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 768, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 0,
                   'Weight for L2 loss on embedding matrix.')  # 5e-4
flags.DEFINE_integer('early_stopping', 20,
                     'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

## Input Features setup

### Node Feature is visual embedding

In [ ]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*2048)
    feature += id_density_dict[l][str(node)]
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),85):
    feature = np.array([0.0]*2048)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [ ]:
import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*2048)
    feature += id_density_dict[l][str(node)]
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),85):
    feature = np.array([0.0]*2048)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

### Node embedding is Text Density Features

In [ ]:

# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*768)
    feature += np.array(positionalencoding1d(768,[id_density_dict[l][str(node)]])[0])
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),85):
    feature = np.array([0.0]*768)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])


In [ ]:

import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*768)
    feature += np.array(positionalencoding1d(768,[id_density_dict[l][str(node)]])[0])
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),85):
    feature = np.array([0.0]*768)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

In [ ]:
print(len(features_train))

30000


### Sparse to tuple

In [ ]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape
    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)
    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)
for l in features_train:
  features_train[l] = preprocess_features(features_train[l])
  

串流輸出內容已截斷至最後 5000 行。
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipyk

In [ ]:
for l in features_eval:
  features_eval[l] = preprocess_features(features_eval[l])

## input setting

In [ ]:
#define input dictionary
# num_obj_dict was defined to record the number of object in each document page
#
y_train = {}
train_mask = {}
for f in new_train_list_dict:
  y_train[f] = np.array([[0]*num_class]*adj_train[f].shape[0])
  y_train[f] = np.array(labels_one_hot_train[f])
  train_mask[f] = [False]*adj_train[f].shape[0]
  train_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]

In [ ]:
y_val = {}
val_mask = {}
for f in new_eval_list_dict:
  y_val[f] = np.array([[0]*num_class]*adj_eval[f].shape[0])
  y_val[f] = np.array(labels_one_hot_eval[f])
  val_mask[f] = [False]*adj_eval[f].shape[0]
  val_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]

In [ ]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)
support_train = {}
for f in adj_train:
  support_train[f] = [preprocess_adj(adj_train[f])]

In [ ]:
support_eval = {}
for f in adj_eval:
  support_eval[f] = [preprocess_adj(adj_eval[f])]

In [ ]:
# just change the number of support gpu
num_supports = 1
tf.compat.v1.disable_eager_execution()
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape = (6,2048)),
    'labels': tf.placeholder(tf.float32, shape=(None, 6)),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0.5, shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32)
}


In [ ]:
# define input dimension which is same as node feature size
model = GCN(placeholders, input_dim=2048, logging=True)

Tensor("graphconvolution_4/SparseTensorDenseMatMul/SparseTensorDenseMatMul:0", shape=(None, 6), dtype=float32)


In [ ]:
# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels, model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], outs_val[2], outs_val[3], outs_val[4], outs_val[5]


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

In [ ]:
def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support'][i]: support[i]
                      for i in range(len(support))})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict

## Train GCN model

In [ ]:
# Train model
for epoch in range(FLAGS.epochs):
    print(epoch)
    output_dic_test = {}
    # Construct feed dictionary
    if epoch == FLAGS.epochs-1:
        output_dic_train = {}
    loss = 0
    for f in features_train:

        feed_dict = construct_feed_dict(
            features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
        outs = sess.run([model.opt_op, model.loss, model.accuracy,
                         model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict)
        loss+=outs[1]
    print(loss)

0
5315.24207158923
1
4762.155317900844
2
4605.174600615376
3
4554.74837380086
4
4560.930261828316
5
4557.314331390124
6
4499.178415675002
7
4507.188373676476
8
4512.386261123685
9
4489.049545446883


In [ ]:
!mkdir /content/drive/MyDrive/Publaynet/final_visual_gcn_trained_model

mkdir: cannot create directory ‘/content/drive/MyDrive/Publaynet/final_visual_gcn_trained_model’: File exists


In [ ]:
saver = tf.train.Saver()
saver.save(sess, '/content/drive/MyDrive/Publaynet/visual_gcn_trained_model_final/visual_gcn_768')

'/content/drive/MyDrive/Publaynet/visual_gcn_trained_model_final/visual_gcn_768'

In [ ]:
ckpt = tf.train.get_checkpoint_state("/content/drive/MyDrive/Publaynet/visual_gcn_trained_model2")
output_dic_test = {}
with tf.Session() as sess:
  saver=tf.train.import_meta_graph('/content/drive/MyDrive/Publaynet/visual_gcn_trained_model2/visual_gcn_768.meta')
  saver.restore(sess, ckpt.all_model_checkpoint_paths[0])
  sess.run(tf.global_variables_initializer())
  for f in features_train:
    cost, acc, pred, labels, emb1, emb2 = evaluate(features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
    output_dic_test[f] = emb1
print("Optimization Finished!")
print(ckpt)

#print(sess.run(accuracy, feed_dict={x: mnist.test.images,y_: mnist.test.labels}))

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Publaynet/visual_gcn_trained_model2/visual_gcn_768
Optimization Finished!
model_checkpoint_path: "/content/drive/MyDrive/Publaynet/visual_gcn_trained_model2/visual_gcn_768"
all_model_checkpoint_paths: "/content/drive/MyDrive/Publaynet/visual_gcn_trained_model2/visual_gcn_768"



In [ ]:
output_dic_test.keys()

dict_keys(['287233', '287535', '286515', '286853', '287465', '287240', '286909', '286867', '286523', '286874', '287931', '287486', '287283', '287304', '287521', '287311', '285152', '287573', '288457', '288756', '287980', '288488', '289054', '211687', '286559', '288763', '288523', '289082', '287367', '288100', '288401', '288770', '289119', '289577', '287727', '288600', '289724', '289606', '289829', '289420', '290045', '289621', '289629', '289492', '288910', '289745', '288917', '289752', '288672', '289935', '289664', '290748', '289671', '289942', '290168', '289696', '289513', '290720', '288995', '289002', '287776', '289527', '290003', '290881', '290551', '289243', '291421', '291132', '290474', '290572', '291890', '287804', '291673', '290923', '291897', '290944', '292483', '291680', '290958', '292279', '291904', '292696', '291052', '292286', '290608', '292703', '292023', '290615', '291911', '292843', '291060', '290629', '292413', '291953', '291067', '292051', '292888', '292420', '291076',

In [ ]:
import pickle
import os 
path = '/content/drive/MyDrive/Publaynet/'
os.chdir(path)
file_name = 'train_0_7_gcn_visual.json' 
with open(file_name,'wb') as file_object:
    pickle.dump(output_dic_test,file_object)

In [ ]:
print(len(output_dic_test['282944'][0]))

768


In [ ]:
output_dic_train_vis = output_dic_train

In [ ]:
# Validation
for f in features_eval:
    cost, acc, pred, labels, emb1, emb2 = evaluate(
        features_eval[f], support_eval[f], y_val[f], val_mask[f], placeholders)
    output_dic_test[f] = emb1
print("Optimization Finished!")

In [ ]:
import pickle
import os 
path = '/content/drive/MyDrive/Publaynet/'
os.chdir(path)
file_name = 'char_density_gcn_emb_train.pkl' 
with open(file_name,'wb') as file_object:
    pickle.dump(output_dic_train,file_object)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Publaynet/train_0_1_gcn_visual.json','rb') as f:
  train_list_dict1 = pickle.load(f)

In [ ]:
output_dic_train_vis = output_dic_train
output_dic_test_vis = output_dic_test

In [ ]:
output_dic_train_den = output_dic_train
output_dic_test_den = output_dic_test

## Matching with Object Classification Dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_text = []
train_label = []
train_density = []
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    tem = train_list_dict[img]['objects'][obj]
    train_text.append(tem['text'])
    train_label.append(tem['category_id'])
  for i in range(len(train_list_dict[img]['objects'].keys())):
    train_density.append(output_dic_train[img][i])

In [ ]:
print(len(train_text))
print(len(train_density))

294480
294480


In [ ]:
df_train = pd.DataFrame(np.array(train_text),columes = ['text'])

In [ ]:
df_train.head()

In [ ]:
df_train.to_pickle('/content/drive/MyDrive/Publaynet/publaynet_gcn-char-density.pkl')
df_test.to_pickle('/content/drive/MyDrive/Publaynet/publayne_gcn-char-density.pkl')